# KMP 算法

> 用于在文本中匹配给定的子串（模式串）
> 假设我们现在有这样一个问题：有一个文本串S，一个模式串P，问P出现在S中吗？如果出现，P在S的什么位置？

## 暴力算法

> 遍历，如果当前遍历到文本位置i，模式串位置j，如果 S[i] == P[j], 则 i+=1, j+=1 继续匹配下一个位置；否则 i = i - (j-1), j = 0，相当于模式串右移一位，从模式串头对齐的位置匹配

In [16]:
# brute force solution
def violent_match(s, p):
    s_len = len(s)
    p_len = len(p)
    i = 0
    while i <= s_len - p_len:
        j = 0
        while j < p_len and s[i] == p[j]:
            i += 1
            j += 1
        
        if j == p_len:
            return i - j
        else:
            i = i - j + 1
    return -1

violent_match("hello", "ll")

2

### 暴力算法的缺点

> 当 S[i] != P[j] 时，暴力算法回退比较 S[i-j+1] 和 P[0]，之前已经知道S[i-j+1] = P[1] (假设 j > 1), 所以等同于比较 P[0] 和 P[1]，这是已知的信息，有没有什么办法利用模式串的信息来优化算法呢

## KMP 算法

> 这里先对 KMP 的流程进行概述，然后再详细解释

> 1. 当 S[i] == P[j] 时，比较S[++i] 和 P[++j]
> 2. 当 S[i] != P[j] 时，比较S[++i] 和 P[next[j-1]]（j>0），或者比较S[++i] 和 P[j]（j==0）
> 3. 当 j == p_len 时，在S中找到模式串的位置i - j, 匹配结束
> 4. 当 i == s_len 时，无匹配，匹配结束


### next 数组

> 不同的实现方式有不同的定义，这里是其中一种定义
> 定义：next 数组的值是对应的子串（比如下标i，对应P[:i+1]）的前缀、后缀中公共元素的最大长度（下文简称最大长度）


### 字符串的前缀、后缀

> 前缀：不包括尾字符，所有包括头字符的子串组合
> 后缀：不包括头字符，所有包括尾字符的子串组合

#### 举个例子
> 字符串：abcab
> 前缀：[abca, abc, ab, a]
> 后缀：[b, ab, cab, bcab]
> next数组
> 1. next[0] 对应的子串是：a，最大长度是：0
> 2. next[1] 对应的子串是：ab，最大长度是：0
> 3. next[2] 对应的子串是：abc，最大长度是：0
> 4. next[3] 对应的子串是：abca，最大长度是：1
> 5. next[4] 对应的子串是：abcab，最大长度是：2


#### next数组实现

> 1. next[0] = 0, j=0（记录前缀的位置），i=1（记录后缀的位置）
> 2. 如果 j>0 且 P[j] != P[i], j=next[j-1]
> 3. 如果 P[j] == P[i], j++
> 4. next[i] = j

In [26]:
# next数组实现
def build_next(p):
    l = len(p)
    next = [0] * l
    j = 0
    for i in range(1, l):
        while j > 0 and p[i] != p[j]:
            j = next[j - 1]
        if p[i] == p[j]:
            j += 1
            next[i] = j
    return next

build_next("ababaca")

[0, 0, 1, 2, 3, 0, 1]

In [20]:
# KMP算法实现
def kmp(s, p):
    s_len = len(s)
    p_len = len(p)
    next = build_next(p)
    j = 0
    for i in range(s_len):
        while j > 0 and s[i] != p[j]:
            j = next[j - 1]
        if s[i] == p[j]:
            j += 1
        if j == p_len:
            return i - j + 1

kmp("hello", "ll")

2

## 扩展

### BM 算法

> 整体流程
> 1. 从模式串的尾部开始比较，初始化：i=0，skip=j=p_len-1
> 2. 如果S[i+p_len-1] == p[j], 比较S[i+-1]和P[--j]
> 3. 否则，S[i+] 称之为坏字符，坏字符规则：P向右移动位数 = 坏字符对应P的位置 - 坏字符在P[:j]中出现的最右位置，如果没有出现，位置则为-1
> 4. 另外一种情况是，当S[i] != P[j]时，后面几位如：S[i+1] = P[j+1] …… S[i+k] = P[j+k] (j+k = p_len-1) 都已经匹配上，S[：i+p_len-1]称为好尾缀，这时可以用好尾缀规则：P向右移动位数 = 好尾缀位置 - 好尾缀在P中上一次出现的位置，如果没有出现，上一次位置则为-1
> 5. 比较坏字符规则和好尾缀规则，取移动位置较大的那个
> 6. P移动后，重新从尾部开始比较。直到完全匹配或者P移动出S的尾部

In [ ]:
def bad_char_heuristic(p):
    l = len(p)
    bc = {}
    for i in range(l):
        bc[p[i]] = i
    return bc